## Install the necessary packages

In [18]:
!pip install openpyxl
!pip install python-dotenv

## Importing the necessary packages

In [20]:
import pandas as pd

In [21]:
import mysql.connector

In [22]:
from dotenv import load_dotenv

## Loading environment variables

In [24]:
load_dotenv()
import os

In [25]:
conn = mysql.connector.connect(
    host=os.getenv('DB_HOST'),
    user=os.getenv('DB_USER'),
    password=os.getenv('DB_PASSWORD'),
    database=os.getenv('DB_NAME')
)
cursor = conn.cursor()
print("CONNECTED")

CONNECTED


# Phase 01 : Extract

## Reading the data from the csv files

In [28]:
products_df = pd.read_csv('products_data.csv')
orders_df = pd.read_csv('orders_data.csv')
users_df = pd.read_csv('users_data.csv')

## Reading the data from the excel files

In [30]:
payments_df = pd.read_excel('payments_data.xlsx')
reviews_df = pd.read_excel('reviews_data.xlsx')

# Phase 02 : Transform

## Fill missing values in each dataframe as needed

In [33]:
products_df = products_df.fillna({
    'Description' : 'No description available',
    'Category' : 'Miscellaneous'
})

In [34]:
orders_df = orders_df.fillna({
    'TotalAmount' : 0.0
})

In [35]:
payments_df = payments_df.fillna({
    'PaymentMethod': 'Unknown',
    'Amount': 0.0,
    'PaymentDate': pd.Timestamp('1970-01-01')
})

In [36]:
reviews_df = reviews_df.fillna({
    'ReviewText': 'no review'
})

## Handling invalid emails in the users DataFrame

In [38]:
def generate_unique_email(index):
    return f'unknown_{index}@example.com'
users_df['Email'] = users_df.apply(lambda row: row['Email'] if '@' in row['Email'] and '.' in row['Email'] else
                                   generate_unique_email(row.name), axis=1)


## Ensure all prices are numeric

In [40]:
products_df['Price'] = pd.to_numeric(products_df['Price'], errors='coerce').fillna(0.0).round(2)

## Ensure all stock quantities are numeric

In [42]:
products_df['StockQuantity'] = pd.to_numeric(products_df['StockQuantity'], errors='coerce').fillna(0).astype(int)

## Standardize and convert date formats to string

In [44]:
orders_df['OrderDate'] = pd.to_datetime(orders_df['OrderDate'], errors='coerce', format='%Y-%m-%d').fillna(pd.Timestamp('1970-01-01'))
orders_df['OrderDate'] = orders_df['OrderDate'].dt.strftime('%Y-%m-%d')

payments_df['PaymentDate'] = pd.to_datetime(payments_df['PaymentDate'], errors='coerce', format='%Y-%m-%d').fillna(pd.Timestamp('1970-01-01'))
payments_df['PaymentDate'] = payments_df['PaymentDate'].dt.strftime('%Y-%m-%d')

# Phase 03: Load

## Delete all records from Payments, Reviews, Orders, Users, and Products tables

In [47]:
tables_to_clear = ['Payments', 'Reviews', 'Orders', 'Users', 'Products']
for table in tables_to_clear:
    cursor.execute(f"DELETE FROM {table}")
conn.commit()

## Insert data into products table

In [49]:
product_id_mapping = {}
for index, row in products_df.iterrows():
    cursor.execute("""
    INSERT INTO Products (ProductName, Description, Price, StockQuantity, Category)
    VALUES (%s, %s, %s, %s, %s) """, (row['ProductName'], row['Description'], row['Price'], row['StockQuantity'], row['Category']))
    product_id_mapping[index] = cursor.lastrowid
conn.commit()

print(len(products_df))

## Insert Data and Map Old UserIDs to new UserIDs

In [52]:
user_id_mapping = {}
for index, row in users_df.iterrows():
    cursor.execute("""
    INSERT INTO Users (UserName, Email, Address, Password)
    VALUES (%s, %s, %s, %s)"""
                   , (row['UserName'], row['Email'], row['Address'], row['Password']))
    user_id_mapping[index + 1] = cursor.lastrowid
conn.commit()

Update Orders Data with New UserIDs

In [54]:
orders_df['UserID'] = orders_df['UserID'].map(user_id_mapping)

## Insert Data into Orders table

In [56]:
order_id_mapping = {}
for index, row in orders_df.iterrows():
    cursor.execute("""
    INSERT INTO Orders (UserID, OrderDate, TotalAmount) 
    VALUES (%s, %s, %s) """, (row['UserID'], row['OrderDate'], row['TotalAmount']))
    order_id_mapping[index] = cursor.lastrowid
conn.commit()

## Map the new OrderIDs to payments_df

In [58]:
payments_df['OrderID'] = payments_df.index.map(order_id_mapping)

## Insert data into Payments table

In [60]:
for index, row in payments_df.iterrows():
    cursor.execute("""
    INSERT INTO Payments (OrderID, PaymentMethod, PaymentDate, Amount)
    VALUES (%s, %s, %s, %s)
    """, (row['OrderID'], row['PaymentMethod'], row['PaymentDate'], row['Amount']))
    conn.commit()

## Map the new UserIDs and ProductIDs to reviews_df

In [62]:
reviews_df['UserID'] = reviews_df['UserID'].map(user_id_mapping)
reviews_df['ProductID'] = reviews_df.index.map(product_id_mapping)

## Filter out rows with invalid ratings

In [64]:
reviews_df['Rating'] = pd.to_numeric(reviews_df['Rating'], errors='coerce')
reviews_df = reviews_df.dropna(subset=['Rating', 'ProductID', 'UserID'])

## Insert data into Reviews table

In [66]:
for index, row in reviews_df.iterrows():
    cursor.execute("""
    INSERT INTO Reviews (ProductID, UserID, Rating, ReviewText)
    VALUES (%s, %s, %s, %s) """, (row['ProductID'], row['UserID'], row['Rating'], row['ReviewText']))
conn.commit()

# Final Verification

In [68]:
def verify_table_count(table_name, expected_count):
    cursor.execute(f"SELECT COUNT(*) FROM {table_name}")
    count = cursor.fetchone()[0]
    if count == expected_count:
        print(f"{table_name} verification passed: {count} records.")
    else:
        print(f"{table_name} verification failed: Expected {expected_count}, but found {count}.")



## Verify Products table

In [70]:
verify_table_count('Products', len(products_df))

Products verification passed: 60 records.


## Verify Users table

In [72]:
verify_table_count('Users', len(users_df))

Users verification passed: 10 records.


## Verify Orders table

In [74]:
verify_table_count('Orders', len(orders_df))

Orders verification passed: 60 records.


## Verify Payments table

In [76]:
verify_table_count('Payments', len(payments_df))

Payments verification passed: 60 records.


## Verify Reviews table

In [78]:
verify_table_count('Reviews', len(reviews_df))

Reviews verification passed: 51 records.
